In [281]:
import pandas as pd 
import numpy as np



In [282]:
ele_2023 = pd.read_csv("Data/Presidential_Elections_Turkey_2023.csv")
ele_2023.head()


,City,Recep Tayyip Erdogan Percentage,R E Votes,Kemal,Kemal Votes
0,Adana,45.96,621003,54.04,730274
1,Adiyaman,68.84,216491,31.16,97982
2,Afyonkarahisar,67.84,312452,32.16,148136
3,Agri,34.66,68790,65.34,129693
4,Amasya,59.16,134428,40.84,92811


In [283]:
ele_2023.info()
duplicate_rows = ele_2023.duplicated()
number_of_duplicates = ele_2023.duplicated().sum()
number_of_duplicates
ele_2023 = ele_2023.rename(columns={"Recep Tayyip Erdogan Percentage": "AK","R E Votes": "AK Votes", "Kemal":"CHP", "Kemal Votes": "CHP Votes"})


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   City                             81 non-null     object 
 1   Recep Tayyip Erdogan Percentage  81 non-null     float64
 2   R E Votes                        81 non-null     int64  
 3   Kemal                            81 non-null     float64
 4   Kemal Votes                      81 non-null     int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 3.3+ KB


In [284]:
print(ele_2023.columns)


Index(['City', 'AK', 'AK Votes', 'CHP', 'CHP Votes'], dtype='object')


In [285]:
ele_2023["AK"] = ele_2023["AK"] / 100
ele_2023["CHP"] = ele_2023["CHP"] / 100
ele_2023

,City,AK,AK Votes,CHP,CHP Votes
0,Adana,0.4596,621003,0.5404,730274
1,Adiyaman,0.6884,216491,0.3116,97982
2,Afyonkarahisar,0.6784,312452,0.3216,148136
3,Agri,0.3466,68790,0.6534,129693
4,Amasya,0.5916,134428,0.4084,92811
...,...,...,...,...,...
76,Yalova,0.5042,85212,0.4958,83799
77,Karabuk,0.6372,98521,0.3628,56084
78,Kilis,0.6999,57337,0.3001,24580
79,Osmaniye,0.6620,212942,0.3380,108743


In [286]:
ele_2024 = pd.read_excel("Data/Local_Election_Turkey_2024.xlsx")


del ele_2024["CHEST"]
del ele_2024["Winner Year"]
del ele_2024["Winner Past Percentage"]
ele_2024
duplicate_rows = ele_2023.duplicated()                 #Check for duplicates
number_of_duplicates = ele_2023.duplicated().sum()
number_of_duplicates 
ele_2024
ele_2024 = ele_2024.rename(columns={"AK Party": "AK", "DEM Party": "DEM" }) #Rename Columns 
ele_2024

turkish_to_latin = {
    'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 
    'ş': 's', 'ü': 'u', 'Ç': 'C', 'Ğ': 'G', 
    'İ': 'I', 'Ö': 'O', 'Ş': 'S', 'Ü': 'U'
}

# Apply the mapping to your dataframe columns
ele_2024['CITY'] = ele_2024['CITY'].replace(turkish_to_latin, regex=True)
ele_2024['Winner Name'] = ele_2024['Winner Name'].replace(turkish_to_latin, regex=True)

ele_2024


#Change the "Oy" to "votes"

columns_to_update = ["AK", "MHP", "CHP", "Prosperity Again", "DEM", "Other"]  

# Replace ' Oy)' with ' votes)' for each specified column
# Replace "Votes" with "votes"
for col in columns_to_update:
    ele_2024[col] = ele_2024[col].str.replace(r' Oy\)', ' votes)', regex=True)
    ele_2024[col] = ele_2024[col].str.replace(r' Votes\)', ' votes)', regex=True)
    
ele_2024.head(1)

,CITY,AK,MHP,CHP,Prosperity Again,DEM,Other,Winner Name,Winner Percentage,Winner Party
0,ADANA,"37.43% (442,486 votes)",0% (0 votes),"46.57% (550,531 votes)","1.86% (22,090 votes)","5.6% (66,221 votes)","8.52% (100,715 votes)",Zeydan Karalar,0.4657,CHP


In [287]:
# Separting percentage from votes, and adding votes to another dataframe
votes = pd.DataFrame()
for col in columns_to_update:
    votes[col] = ele_2024[col].str.extract(r'(\d{1,3}(?:,\d{3})*) votes')[0]
    ele_2024[col] = ele_2024[col].str.replace(r'\s*\(\s*\d{1,3}(?:,\d{3})*\s*votes\s*\)', '', regex=True)
    ele_2024[col] = ele_2024[col].str.replace(r'\s*\([^)]*\)', '', regex=True).str.strip()

    

# Convert 'AK Votes' to numeric, avoiding errors (which turns non-numeric values to NaN)
for col in columns_to_update:
    votes[col] = votes[col].str.replace(',', '')
    votes[col] = pd.to_numeric(votes[col], errors='coerce')
    
#Convert "one%" into "1%"
ele_2024.at[10, "Prosperity Again"] = "1%"

# Converting  "Selahattin Ekicio?lu"...into...."Selahattin Ekicio?lu"
ele_2024.at[49, "Winner Name"] = "Selahattin Ekicioglu"

ele_2024.head()

,CITY,AK,MHP,CHP,Prosperity Again,DEM,Other,Winner Name,Winner Percentage,Winner Party
0,ADANA,37.43%,0%,46.57%,1.86%,5.6%,8.52%,Zeydan Karalar,0.4657,CHP
1,ADIYAMAN,27.42%,0%,49.74%,13.74%,2.65%,6.42%,Abdurrahman Tutdere,0.4974,AK Party
2,AFYONKARAHISAR,31.82%,10.87%,50.73%,2.11%,0.14%,4.3%,Burcu Koksal,0.5073,AK Party
3,PAIN,33.7%,0%,2.57%,4.71%,51.06%,7.94%,Hazal Aras,0.5106,AK Party
4,AKSARAY,36.85%,28.72%,7.79%,2.42%,0.27%,23.93%,Evren Dincer,0.3685,AK Party


In [288]:
# Converting the column to string if it's not then striping the '%' and convert to float

for col in columns_to_update:
    ele_2024[col] = ele_2024[col].astype(str).str.rstrip('%').astype(float) / 100
ele_2024.head()

,CITY,AK,MHP,CHP,Prosperity Again,DEM,Other,Winner Name,Winner Percentage,Winner Party
0,ADANA,0.3743,0.0000,0.4657,0.0186,0.0560,0.0852,Zeydan Karalar,0.4657,CHP
1,ADIYAMAN,0.2742,0.0000,0.4974,0.1374,0.0265,0.0642,Abdurrahman Tutdere,0.4974,AK Party
2,AFYONKARAHISAR,0.3182,0.1087,0.5073,0.0211,0.0014,0.0430,Burcu Koksal,0.5073,AK Party
3,PAIN,0.3370,0.0000,0.0257,0.0471,0.5106,0.0794,Hazal Aras,0.5106,AK Party
4,AKSARAY,0.3685,0.2872,0.0779,0.0242,0.0027,0.2393,Evren Dincer,0.3685,AK Party


In [290]:
#Checking the data set for other errors
ele_2024[50:]

,CITY,AK,MHP,CHP,Prosperity Again,DEM,Other,Winner Name,Winner Percentage,Winner Party
50,REPORT,0.2727,0.2374,0.4197,0.0225,0.0024,0.0451,Hakan Bilecen,0.4197,AK Party
51,KOCAELI,0.4583,0.0000,0.2896,0.0759,0.0326,0.1434,Tahir Buyukakin,0.4583,AK Party
52,KONYA,0.4943,0.0000,0.1277,0.2348,0.0178,0.1252,Ugur Ibrahim Altay,0.4943,AK Party
53,KUTAHYA,0.2770,0.2925,0.2969,0.0807,0.0014,0.0512,Eyup Kahveci,0.2969,MHP
54,MALATYA,0.4896,0.0000,0.3727,0.0938,0.0053,0.0383,Sami Er,0.4896,AK Party
55,MANISA,0.0000,0.2978,0.5725,0.0352,0.0000,0.0943,Ferdi Zeyrek,0.5725,MHP
56,MARDIN,0.2778,0.0000,0.0249,0.0447,0.5740,0.0784,Ahmet Turk,0.5740,HDP
57,MERSIN,0.0000,0.0000,0.3104,0.0117,0.5951,0.0827,Vahap Secer,0.5951,CHP
58,MUGLA,0.3269,0.0000,0.5500,0.0113,0.5500,0.0934,Ahmet Aras,0.5500,CHP
59,MUS,0.2526,0.0000,0.0153,0.1285,0.4288,0.1746,Sirri Soylemez,0.4288,AK Party
